### 8.2 Customizing string formatting

In [1]:
_formats = {
    'ymd': '{d.year}-{d.month}-{d.day}',
    'mdy': '{d.month}/{d.day}/{d.year}',
    'dmy': '{d.day}/{d.month}/{d.year}'
}

In [2]:
class Date:

    def __init__(self, day, month, year):
        self.day , self.month, self.year = day, month, year
        
    def __format__(self, code):
        if code=='':
            code = 'ymd'
        fmt = _formats[code]
        return fmt.format(d=self)

In [3]:
d = Date(31, 7, 1985)
print(format(d, 'dmy'))
print(format(d, 'mdy'))
print('{:ymd}'.format(d))

31/7/1985
7/31/1985
1985-7-31
